In [5]:
import pandas as pd
import numpy as np
import os
import requests
import json
import urllib.request
import re

In [11]:
app_id = '24ac8855'
app_key = 'baa032bb2944756dcf361a18ae7e3ab9'

language = 'en'
word_id = 'Ace'.lower()

url = 'https://od-api.oxforddictionaries.com:443/api/v1/entries/' + language + '/' + word_id.lower()

#with urllib.request.urlopen(url) as url:
    #data = json.loads(url.read().decode())
    #print(data)

actual_dictionary = {}

def find_defs(diction):
    """
    This function executes when we reach a list containing dictionary data strucutres that may contain several different
    definitions for a given word. It runs through this list, for each list finding keys in the dictionary data
    structure containing the string "definitions" or "short_definitions" and appending the corresponding value 
    (a string) of this key to a master string, which we call stringy. At the end of this function, we use the 
    built-in function set() on the list given to us by the .split() method. The reasons for a set are:
    1) checking membership in sets is efficient in python
    2) sets do not contain duplicate elements, so we are using less space.
    """
    stringy = ''
    for i in diction:
        for k, v in i.items():
            if k == 'definitions' or k == 'short_definitions':
                stringy += v[0]
    return set(stringy.split(' '))


def find_definition(word):
    """
    This function takes in a word as input and creates an entry in our dataset consisting of a word-definition pair.
    """
    url = 'https://od-api.oxforddictionaries.com:443/api/v1/entries/en/' + word.lower()
    # there may be times while looking up words that the dictionary does not contain a word for whatever reason.
    # this chunk of code catches this error.
    try:
        r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
        dicty = r.json()
        
        # this process is ugly, but it works. 
        # The structure of the data returned from the API call is a bit awkward.
        first_layer = dicty['results']
        second_layer = first_layer[0]
        third_layer = second_layer['lexicalEntries']
        # third layer contains multiple definitions or "senses" of the word
        fourth_layer = third_layer[0]
        fifth_layer = fourth_layer['entries']
        sixth_layer = fifth_layer[0]
        seventh_layer = sixth_layer['senses']
        actual_dictionary[word] = find_defs(seventh_layer)
    except json.JSONDecodeError:
        pass
    
word_list = ['This', 'chapter', 'focuses']
for word in word_list:
    find_definition(word)

def find_contents(diction):
    try:
        for k, v in diction.items():
            print(k, '\n', v, '\n\n')
    except AttributeError:
        for i in diction:
            print(i, '\n\n')
            
find_contents(actual_dictionary)

This 
 {'hand', 'mentionedreferring', 'mentioned', 'being', 'close', 'just', 'a', 'handreferring', 'used', 'at', 'to', 'specific', 'person', 'identify', 'or', 'indicated', 'experiencedused', 'thing'} 


chapter 
 {'main', 'a', 'societylocal', 'or', 'distinctive', 'period', 'lifeperiod', 'booka', 'orderreligious', 'body', 'division', 'with', 'knightly', 'religious', 'group', 'in', 'typically', 'community', 'number', 'history', 'bodya', 'timethe', 'titledivision', "person's", 'branch', 'governing', 'local', 'book,', 'of'} 


